In [1]:
import io
import os
from pathlib import Path
import requests
import zipfile

import pandas as pd
import synapseclient
from synapseclient import Activity, Project, Folder, File

syn = synapseclient.Synapse()
syn.login()

syn_project = syn.get(Project(name='mHealthFeaturization'))
dpath = lambda s: os.path.join(data_dir, s)

# set the user's home directory as the data directory
data_dir = os.path.join(str(Path.home()), '.mhealth')
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

Welcome, Luke Waninger!



### Get the national zipcode listing

In [2]:
zname = '2017_national_zipcodes.csv'
url =  'http://www2.census.gov/geo/docs/maps-data/data/gazetteer/2017_Gazetteer/2017_Gaz_zcta_national.zip'

# download the file from the US Census Bureau
response = requests.get(url)
if response.ok:
    t_path = dpath('2017_Gaz_zcta_national.txt')
    f = io.BytesIO(response.content)

    with zipfile.ZipFile(f, 'r') as f:
        f.extractall(path=data_dir)

    t = pd.read_csv(t_path, sep='\t')
    t.columns = [s.strip() for s in t.columns]
    t = t.loc[:, ['GEOID', 'INTPTLAT', 'INTPTLONG']]
    t.columns = ['zipcode', 'lat', 'lon']

    t.to_csv(dpath(zname), index=None)
    os.remove(t_path)

    sfile = syn.setProvenance(
        syn.store(File(name=zname, path=dpath(zname), parent=syn_project)),
        activity=Activity(
            used=[dict(name='US Census Bureau Gaz', url=url)]
            executed=[di]
        )
    )


##################################################
 Uploading file to Synapse storage 
##################################################

